In [109]:
import numpy as np
import matplotlib.pyplot as plt
from graphsage.utils2 import load_cora, load_ppi, load_wikics, custom_load_pubmed

In [3]:
# import sys
# import os
  
# # getting the name of the directory
# # where the this file is present.
# current = os.path.dirname(os.path.realpath("DatasetAnalysis.ipynb"))
  
# # Getting the parent directory name
# # where the current directory is present.
# parent = os.path.dirname(current)
  
# # adding the parent directory to 
# # the sys.path.
# #sys.path.append(parent)
# # print(parent)

In [121]:
np.random.seed(1)
rand_indices = np.random.permutation(2708)
rand_indices

array([ 996, 1140, 1145, ..., 1096,  235, 1061])

In [5]:
data_dic = load_cora({})

In [6]:
data_dic.keys()

dict_keys(['feat_data', 'labels', 'adj_lists', 'train_mask', 'test_mask', 'val_mask', 'distances', 'cluster_labels', 'freq', 'dist_in_graph', 'centralityev', 'centralitybtw', 'centralityh', 'centralityd', 'lsh_neighbour_list'])

In [10]:
data_dic['feat_data'].shape

(2708, 1433)

In [11]:
data_dic['labels'].shape

(2708, 1)

In [86]:
def get_centroid(feat_data):
    return feat_data.mean(axis=0)
def get_mean_distance_from_centroid(points,distance_type='euclidean'):
    distance_ = cdist(points,get_centroid(points).reshape(1,-1),metric=distance_type).mean(axis=0)[0]
    return distance_
def get_distance_to_other_centroids(points,centroids_mod,distance_type='euclidean',measure='nearest'):
    dist_mat = cdist(points,centroids_mod,metric=distance_type)
    if measure == 'nearest':
        interdist = dist_mat.min(axis=1).mean()
    elif measure == 'mean':
        interdist = dist_mat.mean(axis=1).mean()
    else:
        print('specify correct measure')
        return 0
    return interdist

In [17]:
sample_data = np.random.random((10,20))
get_centroid(sample_data).shape

(20,)

In [34]:
from scipy.spatial.distance import cdist
cdist(sample_data,get_centroid(sample_data).reshape(1,-1)).mean(axis=0)

array([1.21868423])

In [25]:
from scipy.spatial import distance
distance.euclidean(sample_data[5],get_centroid(sample_data))

1.1162463070332478

In [33]:
get_mean_distance_from_centroid(sample_data)

array([1.21868423])

In [35]:
sample1=np.random.random((10,20))
sample2=np.random.random((10,20))
centroid1,centroid2 = get_centroid(sample1),get_centroid(sample2)

In [51]:
cdist(sample1,np.array([centroid1,centroid2]),metric='euclidean').mean(axis=1).mean()

1.2709469765996286

In [39]:
cdist(sample1,centroid2.reshape(1,-1))

array([[1.50771416],
       [1.22161472],
       [1.39401183],
       [1.31493144],
       [1.44248919],
       [1.19471073],
       [1.44638972],
       [1.31527031],
       [1.02692004],
       [1.42214245]])

In [100]:
get_distance_to_other_centroids(sample2,np.array([centroid1]),measure='nearest')

1.3647179032374468

In [90]:
a=[1,2,3]
a.pop(1)
print(a)
np.zeros(10)

[1, 3]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [60]:
sorted(list(np.unique(data_dic['labels'])))

[0, 1, 2, 3, 4, 5, 6]

In [107]:
def get_ratio(list1,list2):
    temp=[]
    for i in range(len(list1)):
        temp.append(list1[i]/list2[i])
    return temp
#get_ratio(mean_distance_owncentroid,mean_distance_othercentroid)

In [125]:
def give_distance_statistics(feat_data, labels, measure='nearest',distance_type='euclidean'):
    feat_data = np.array(data_dic['feat_data'])
    labels = np.array(data_dic['labels'])
    mean_distance_owncentroid = []
    mean_distance_othercentroid = []
    centroids = []
    for i in sorted(list(np.unique(labels))):
        centroids.append(get_centroid(feat_data[(labels == i).reshape(-1,)]).tolist())
    for i in sorted(list(np.unique(labels))):
        temp = centroids.copy()
        temp.pop(i)
        mean_distance_owncentroid.append(get_mean_distance_from_centroid(feat_data[(labels == i).reshape(-1,)],distance_type=distance_type))
        mean_distance_othercentroid.append(get_distance_to_other_centroids(feat_data[(labels == i).reshape(-1,)],temp,distance_type=distance_type,measure=measure))
        if i == 8:
            print(mean_distance_othercentroid)
    return get_ratio(mean_distance_owncentroid,mean_distance_othercentroid)

In [75]:
feat_data[(labels == 0).reshape(-1,)].shape

(818, 1433)

In [76]:
(labels == 0).reshape(-1,).sum()

818

In [102]:
mean_distance_owncentroid

[3.9880151440213134,
 4.108184312386862,
 4.071730872232723,
 4.060509463212485,
 4.042021659329825,
 3.9244409164180585,
 4.067456678249878]

In [103]:
mean_distance_othercentroid

[4.035900361897457,
 4.166184735701382,
 4.146997789248885,
 4.11969111078311,
 4.087108511146307,
 4.015141303057532,
 4.1391663917048485]

### Cora

In [110]:
data_dic = load_cora(lsh_helper={})

In [113]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels']))

0
1
2
3
4
5
6
[0.9881351833340032, 0.9860782881715504, 0.9818502635300912, 0.9856344454039965, 0.988968520974297, 0.9774104122885028, 0.9826753247710261]


In [114]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean'))

0
1
2
3
4
5
6
[0.9692983317758183, 0.9683800747977145, 0.9633841548617899, 0.9663594993491936, 0.972334162186959, 0.9601267087329962, 0.967542561617903]


In [115]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean',distance_type='cosine'))

0
1
2
3
4
5
6
[0.8937097653366343, 0.8704860321659481, 0.848904939208603, 0.8798365281305003, 0.8898270789309762, 0.8487707097839141, 0.8703702645561137]


In [116]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='nearest',distance_type='cosine'))

0
1
2
3
4
5
6
[0.9536853496383274, 0.9380107860359165, 0.9244451449036671, 0.9387265683905869, 0.9550732841329769, 0.9075683141726592, 0.923312263347089]


## WikiCS

In [123]:
data_dic = load_wikics(lsh_helper={})

Loading WikiCS
loading dataframe of
Loading normal random walks
loading khop neighbours


In [126]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels']))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean'))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean',distance_type='cosine'))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='nearest',distance_type='cosine'))

[1.283969808161322, 1.282474237865921, 1.1821322283371603, 1.3267944629062163, 1.2518883415848734, 1.2694322883897027, 1.2282115901627089, 1.1882865666530333, 1.226803136828503]
[0.8828396447395558, 0.9988908937437361, 1.0130244160135158, 0.9377665672085207, 0.9919740757645626, 0.9192803341002034, 0.8780083788814119, 0.9702896024873485, 1.000099628138771, 0.9634512160207751]
[1.5272010746458093, 1.4913604673648104, 1.4061397249453949, 1.5348433766507712, 1.4727224957939533, 1.4385205816738629, 1.4424372584743985, 1.410394836939244, 1.4464607065447967]
[0.7422332711207326, 0.858982027281967, 0.8516428268239928, 0.8106517627784886, 0.8432279564892518, 0.8112251941717733, 0.7476096869146847, 0.8174888833973342, 0.8482258490605364, 0.8203066469827547]
[0.16402085037121006, 0.14064909445721105, 0.13626012863638032, 0.17594823406835172, 0.14340024727573722, 0.13600415767826182, 0.13110873601035836, 0.1325272108756037, nan]
[0.5510849901947928, 0.7314336042691885, 0.7216309161667189, 0.678406

## Pubmed

In [119]:
data_dic = custom_load_pubmed(lsh_helper={})

In [120]:
print(give_distance_statistics(data_dic['feat_data'],data_dic['labels']))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean'))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='mean',distance_type='cosine'))

print(give_distance_statistics(data_dic['feat_data'],data_dic['labels'],measure='nearest',distance_type='cosine'))

0
1
2
[0.9665461569992628, 0.9908422105892141, 0.9781508772120524]
0
1
2
[0.9605801211430621, 0.97169988168909, 0.9582733954288206]
0
1
2
[0.828178198162361, 0.8966486642092393, 0.868239573981817]
0
1
2
[0.8522521899657841, 0.9572069469908958, 0.9153486606916252]
